# Loading additional context item fields
The currently embedded sdk does not load all the properties/fields of context items of a ContextHub view. This workaround shows how to extract all properties/fields defined in a ContextHub view into a DataFrame by using API calls to retrieve the context items as a Pandas DataFrame.
As input for the function you will need to provide the ContextHub view ID, you can acquire it by following these steps:
1. Click on the blue plus button on the top left of the screen, this will open a work organizer side window on the right.
2. Select the ContextHub view you want to load and click on open.
3. The ContextHub view is now available as a snippet in the left sidebar under, by clicking on it you will add it to the notebook.
4. You can now copy the uuid of the ContextHub view and use it as input for the `fetch_context_items_by_view`function.


In [ ]:
import os
import requests
import pandas as pd
from datetime import datetime

def fetch_context_items_by_view(view_id: str) -> pd.DataFrame:
    """
    Fetch all context items with all context fields defined by a saved Context-Hub view.

    Parameters
    ----------
    view_id : str
        UUID of the Context-Hub view whose filters you want to apply.

    Returns
    -------
    pd.DataFrame
        One row per context item, with columns:
        - ``created`` : datetime parsed from ``createdDate``
        - ``key``     : the ``shortKey`` of the item
        - ``state``   : the last event state
        - all additional ``fields`` flattened into individual columns
    """
    # ensure environment variables
    base = os.environ.get("KERNEL_SERVER_URL")
    token = os.environ.get("KERNEL_USER_TOKEN")
    if not base or not token:
        raise EnvironmentError("KERNEL_SERVER_URL and KERNEL_USER_TOKEN must be set")

    # 1. fetch view filters
    view_url = f"{base}/context/view/{view_id}/enriched"
    headers = {"Authorization": f"Bearer {token}"}
    resp = requests.get(view_url, headers=headers)
    resp.raise_for_status()
    filters = resp.json().get("data", {}).get("filters", [])
    if not isinstance(filters, list):
        raise ValueError("Unexpected view JSON: 'data.filters' missing or not a list")

    # 2. page through search results
    search_url = f"{base}/context/v2/item/search"
    headers["Content-Type"] = "application/json"
    payload = {"filters": filters, "fetchSize": 1000, "continuationToken": None}

    items = []
    while True:
        r = requests.post(search_url, headers=headers, json=payload)
        r.raise_for_status()
        page = r.json()

        content = page.get("content", [])
        if not content:
            break

        for it in content:
            items.append({
                "created": datetime.strptime(it["createdDate"], "%Y-%m-%dT%H:%M:%S.%fZ"),
                "key":     it["shortKey"],
                "state":   it["events"][-1].get("state"),
                **it.get("fields", {})
            })

        token_next = page.get("page", {}).get("continuationToken")
        payload["continuationToken"] = token_next

    return pd.DataFrame(items)

In [ ]:
df = fetch_context_items_by_view("your ContextHub view ID")
df